# Tarefa 3: Explorar e consultar dados do kernel PySpark

Neste notebook, você vai:

- Aprender a usar um notebook do Studio para procurar visualmente um cluster do Amazon EMR, autenticar-se nele e conectar-se a ele.
- Explorar e consultar os dados usando o Spark ML. 
- Monitorar os trabalhos do Spark com a IU do Spark.

### Tarefa 3.1: Informações da sessão

Como você está usando o kernel do PySpark, é possível usar a mágica %%info do PySpark para exibir as informações da sessão atual.

In [ ]:
%%info

### Tarefa 3.2: Explorar e consultar os dados

Quando o kernel do PySpark é usado, um SparkContext e um HiveContext são criados automaticamente após a conexão com um cluster do EMR. Você pode usar o HiveContext para consultar dados na tabela Hive e disponibilizá-los em um dataframe do Spark.

Use o HiveContext para consultar o Hive e observar os bancos de dados e as tabelas.

In [ ]:
#query-hive

sqlContext = HiveContext(sqlContext)

dbs = sqlContext.sql("show databases")
dbs.show()

tables = sqlContext.sql("show tables")
tables.show()

Consulte a tabela de dados de adultos e coloque os dados em um dataframe do Spark.

In [ ]:
#load-data

adult_df = sqlContext.sql("select * from adult_data").cache()

Use o dataframe para consultar a forma e visualizar as cinco primeiras linhas do conjunto de dados.

In [ ]:
#view-shape

print((adult_df.count(), len(adult_df.columns)))

#Show first 5 rows 
adult_df.head(5)

Para gerar uma saída mais limpa, converta o dataframe do Spark em um dataframe do Pandas.

In [ ]:
#convert-dataframe

adult_df.limit(5).toPandas()

Alguns algoritmos de machine learning (ML), como regressão linear, exigem recursos numéricos. O conjunto de dados de adultos usado neste laboratório inclui recursos categóricos como **workclass**, **education**, **occupation**, **marital status**, **relationship**, **race** e **sex**.

O bloco de código a seguir ilustra como usar o StringIndexer e o OneHotEncoderEstimator para converter variáveis categóricas em um conjunto de variáveis numéricas que usa os valores 0 e 1.

- O StringIndexer converte uma coluna de valores de string em uma coluna de índices de rótulo. 
- O OneHotEncoderEstimator mapeia uma coluna de índices de categoria para uma coluna de vetores binários, com no máximo um "1" em cada linha que indica o índice de categoria dessa linha.

A codificação one-hot no Spark é um processo de duas etapas. Primeiro você usa o **StringIndexer**, depois o **OneHotEncoder**.

Consulte [StringIndexer](https://spark.apache.org/docs/latest/ml-features.html#stringindexer) e [OneHotEncoder](https://spark.apache.org/docs/latest/ml-features.html#onehotencoder) para saber mais.

In [ ]:
#convert-variables

from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator

categorical_variables = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']

indexers = [StringIndexer(inputCol=column, outputCol=column+"-index") for column in categorical_variables]

encoder = OneHotEncoderEstimator(
    inputCols=[indexer.getOutputCol() for indexer in indexers],
    outputCols=["{0}-encoded".format(indexer.getOutputCol()) for indexer in indexers]
)

A classe VectorAssembler usa várias colunas como entrada. Ela gera uma só coluna que contém uma matriz de valores.

Consulte [VectorAssembler](https://spark.apache.org/docs/latest/ml-features.html#vectorassembler) para saber mais sobre esse assembler.

In [ ]:
#vector-assembler

from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=encoder.getOutputCols(),
    outputCol="categorical-features"
)

Um pipeline é uma lista ordenada de transformadores e estimadores. É possível definir um pipeline para automatizar e garantir a repetição das transformações a serem aplicadas a um conjunto de dados. Nesta etapa, você define o pipeline e depois o aplica ao conjunto de dados.

Semelhante ao **StringIndexer**, um pipeline é um **estimator**. O método pipeline.fit() retorna um **PipelineModel**, que é um **transformador**.

Consulte [Pipelines](https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline) para saber mais sobre os pipelines de machine learning.

In [ ]:
#pyspark-pipelines

from pyspark.ml import Pipeline

# Define the pipeline based on the stages created in previous steps.
pipeline = Pipeline(stages=indexers + [encoder, assembler])

# Define the pipeline model.
pipelineModel = pipeline.fit(adult_df)

# Apply the pipeline model to the dataset.
adult_df = pipelineModel.transform(adult_df)

Examine todas as diferentes colunas que foram criadas na etapa anterior.

In [ ]:
#print-schema

adult_df.printSchema()

Após a aplicação das transformações, uma só coluna conterá uma matriz com todas as variáveis categóricas codificadas.

In [ ]:
#view-categorical-features

adult_df.select('categorical-features').show(truncate=False)

Agora, codifique o rótulo de destino.

In [ ]:
#encode-target

indexer = StringIndexer(inputCol='income', outputCol='label')

adult_df = indexer.fit(adult_df).transform(adult_df)

### Tarefa 3.3: Monitorar e depurar com a IU do Spark

Nesta seção, você vai usar a IU do Spark para monitorar e inspecionar o desempenho dos trabalhos do Spark executados na etapa anterior.

Obtenha as informações da sessão atual do Spark.

In [ ]:
%%info

Encontre os hiperlinks da **IU do Spark** e do **Log do driver**. Neste laboratório, o link **Log do driver** está inativo. O URL pré-assinado da **IU do Spark** é gerado no momento da conexão com o cluster do EMR. Ao clicar nesse link, você acessará a IU do Spark para inspecionar as execuções de trabalho do Spark em um navegador da web. Essas métricas ajudam no ajuste de desempenho. 

Veja alguns recursos importantes a serem procurados no servidor Spark:
- A aba **Jobs** (Trabalhos) mostra o status de todos os trabalhos do Spark nessa aplicação do Spark.
- Na seção de resumo, a seção **Event Timeline** (Linha do tempo de eventos) mostra os vários estágios da execução.
- A seção **Completed Jobs** (Trabalhos concluídos) é mostrada em formato tabular. Na seção **Completed Jobs** (Trabalhos concluídos), você pode escolher um trabalho para examinar as informações sobre os estágios de tarefas que ele contém.
- Usando a **DAG Visualization** (Visualização DAG), você pode explorar as tarefas que já foram executadas. Como na visualização **Event Timeline** (Linha do tempo de eventos), usando a **DAG Visualization** (Visualização DAG), você pode escolher um estágio e expandir os detalhes que ele contém.

### Conclusão

Parabéns! Você se conectou a um cluster do EMR e explorou e consultou os dados usando o PySpark com sucesso.

## Limpeza

Você concluiu este notebook. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de notebook.
- Retorne à sessão do laboratório e continue com a **Conclusão**.